In [78]:
import pandas as pd
import pandasql as ps
from pandasql import sqldf
import datetime
import os

In [79]:
filepath = "C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I"

In [80]:
#curr_dir = os.getcwd()
#nicecxone_callanalytics_fp = f"{curr_dir}\\nicecxone_callanalytics.parquet"

In [81]:
lst_parquet_files = os.listdir(filepath +'/mockfiles')
lst_parquet_files,len(lst_parquet_files)

(['API_Logs.parquet',
  'employees.parquet',
  'message_information.parquet',
  'multi_availability.parquet',
  'multi_response.parquet',
  'nicecxone_callanalytics.parquet',
  'nicecxone_customersatisfaction.parquet',
  'postout_bulk.parquet',
  'ppa_cancellationnotification.parquet',
  'request_information.parquet',
  'servicenow_applicationcancellation.parquet',
  'servicenow_applicationrejection.parquet',
  'servicenow_bonding.parquet',
  'servicenow_caseclosed.parquet',
  'servicenow_casecreated.parquet',
  'servicenow_casereopened.parquet',
  'servicenow_changeincidents.parquet',
  'servicenow_changerecordclosed.parquet',
  'servicenow_changerecordopened.parquet',
  'servicenow_changerecordupdated.parquet',
  'servicenow_complaintresponse.parquet',
  'servicenow_deployedfixes.parquet',
  'servicenow_firstcontactresolution.parquet',
  'servicenow_foscomplaint.parquet',
  'servicenow_incidentclosed.parquet',
  'servicenow_incidentopened.parquet',
  'servicenow_permfixavailable.parq

In [82]:
## calling in all dataframes

df_API_Logs = pd.read_parquet(filepath+'/mockfiles/API_Logs.parquet')
df_servicenow_changeincidents = pd.read_parquet(filepath+'/mockfiles/servicenow_changeincidents.parquet')


In [83]:
## converting the date column to proper format

df_API_Logs['message_call_timestamp']=pd.to_datetime(df_API_Logs['message_call_timestamp'], infer_datetime_format=True)
df_API_Logs['message_recieved_timestamp']=pd.to_datetime(df_API_Logs['message_recieved_timestamp'], infer_datetime_format=True)

df_servicenow_changeincidents['date_of_incident']=pd.to_datetime(df_servicenow_changeincidents['date_of_incident'],infer_datetime_format=True)
df_servicenow_changeincidents['date_of_change']=pd.to_datetime(df_servicenow_changeincidents['date_of_change'],infer_datetime_format=True)

In [84]:
df_API_Logs.to_parquet('C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I/mockfiles/API_Logs.parquet')
df_servicenow_changeincidents.to_parquet('C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I/mockfiles/servicenow_changeincidents.parquet')

### KPI 5.1
Document Creation

#### today-30 to today
#### API_description = 'Document_View'
#### s1.count records
#### s2.time_to_view = message_received_timestamp - message_call_timestamp
#### s3. count where time_to_view <= 2000MS
#### s4. s3/s1*100

In [85]:
df_API_Logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   API_ID                      300 non-null    object        
 1   API_description             300 non-null    object        
 2   message_call_timestamp      300 non-null    datetime64[ns]
 3   message_recieved_timestamp  300 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 9.5+ KB


In [128]:
sql_query = """


WITH selected_df AS (
    SELECT *
    ,strftime('%s', message_recieved_timestamp) - strftime('%s', message_call_timestamp) AS time_to_view
    FROM df_API_Logs
    WHERE API_description ="Document View" 
    AND (message_call_timestamp BETWEEN DATETIME('NOW','-30 days') AND DATETIME('NOW'))
)
,
agg_results AS (
SELECT COUNT(*) AS Total
,SUM(CASE WHEN time_to_view <= 2 THEN 1 ELSE 0 END) AS sub_2_count
FROM selected_df
)

SELECT (sub_2_count * 1.0 / Total) * 100 AS PC_Sub_2
FROM agg_results


"""

print(sqldf(sql_query, locals()))

  PC_Sub_2
0     None


In [35]:
df_API_Logs

,API_ID,API_description,message_call_timestamp,message_recieved_timestamp
0,API001,Payment Gateway API,2024-06-04 03:48:06,2024-06-04 03:48:10
1,API002,User Management API,2024-02-24 13:05:40,2024-02-24 13:05:44
2,API003,Payment Gateway API,2024-11-03 21:18:09,2024-11-03 21:18:12
3,API004,User Management API,2024-06-01 14:37:32,2024-06-01 14:37:33
4,API005,Product Catalog API,2024-09-02 12:32:50,2024-09-02 12:32:51
...,...,...,...,...
295,API296,Product Catalog API,2024-04-15 17:51:04,2024-04-15 17:51:08
296,API297,Payment Gateway API,2024-01-21 16:26:41,2024-01-21 16:26:42
297,API298,User Management API,2024-05-03 22:26:50,2024-05-03 22:26:51
298,API299,Product Catalog API,2024-01-02 09:28:13,2024-01-02 09:28:16


### KPI 11.11
Post Release Incident Rate
No releases shall cause a P1 or P2 incident in the live environment

#### for today-30 to today
#### count where incident_Priority- = 1
#### count where incident_Priority = 2

In [129]:
df_servicenow_changeincidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   incident_id        100 non-null    object        
 1   change_id          100 non-null    object        
 2   date_of_incident   100 non-null    datetime64[ns]
 3   date_of_change     100 non-null    datetime64[ns]
 4   incident_priority  100 non-null    int64         
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 4.0+ KB


In [140]:
sql_query = """

SELECT
    (SELECT COUNT(*) FROM df_servicenow_changeincidents WHERE incident_Priority =1 AND date_of_incident >= DATE('now', '-30 days')) AS priority_1_count,
    (SELECT COUNT(*) FROM df_servicenow_changeincidents WHERE incident_Priority =2 AND date_of_incident >= DATE('now', '-30 days')) AS priority_2_count;

"""

print(sqldf(sql_query, locals()))

   priority_1_count  priority_2_count
0                 0                 0


In [131]:
df_servicenow_changeincidents

,incident_id,change_id,date_of_incident,date_of_change,incident_priority
0,INC25,CHG99,2023-12-20 14:39:00,2023-12-21 14:39:00,1
1,INC86,CHG55,2023-12-02 06:01:00,2023-12-03 06:01:00,1
2,INC49,CHG13,2023-12-19 00:17:00,2023-12-20 00:17:00,1
3,INC23,CHG8,2023-12-02 01:27:00,2023-12-03 01:27:00,1
4,INC2,CHG22,2023-12-07 02:20:00,2023-12-08 02:20:00,2
...,...,...,...,...,...
95,INC32,CHG62,2023-12-18 07:47:00,2023-12-19 07:47:00,2
96,INC64,CHG21,2023-12-11 23:07:00,2023-12-12 23:07:00,1
97,INC8,CHG47,2023-12-12 19:01:00,2023-12-13 19:01:00,1
98,INC55,CHG79,2023-12-01 02:02:00,2023-12-02 02:02:00,1
